In [2]:
import os
import urllib.request
from tqdm import tqdm
import pandas as pd
import gzip

Download Data

In [ ]:
# download raw datafile to /dataset

# https://stackoverflow.com/a/53877507/12802401 (Chris Chute)
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_file(url, filename):
    if not os.path.exists('dataset'):
        os.makedirs('dataset')
    if not os.path.exists(filename):
        print('Downloading', url)
        download_url(url, filename)
    else:
        print('File exists:', filename)

In [2]:
review_url = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-California.json.gz'
metadata_url = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/meta-California.json.gz'

review_zip_file = 'dataset/review-California.json.gz'
metadata_zip_file = 'dataset/meta-California.json.gz'

review_file = 'dataset/review-California.json.gz'
metadata_file = 'dataset/meta-California.json.gz'

In [ ]:
download_file(review_url, review_zip_file)
download_file(metadata_url, metadata_zip_file)

unzip

In [ ]:
# unzip the files
def unzip_file(zip_file, output_file):
    if not os.path.exists(output_file):
        print('Unzipping', zip_file)
        with gzip.open(zip_file, 'rb') as f:
            with open(output_file, 'wb') as f_out:
                f_out.write(f.read())
    else:
        print('File exists:', output_file)

unzip_file(review_file, review_file)
unzip_file(metadata_file, metadata_file)

read in meta data to pandas

In [3]:
metadata_df = pd.read_json(metadata_file, lines=True, encoding='utf-8')

filter out the restaurants in Los Angeles

In [10]:
# filter out all restaurants in la
indices = metadata_df['address'].str.contains('Los Angeles', case=False, na=False)
la_metadata_df = metadata_df[indices]

la_metadata_df.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
34,Beads and More,"Beads and More, 800 Maple Ave # 101, Los Angel...",0x80c2c6349f4c5123:0x847fe6c3b420dff8,None,34.040586,-118.251314,"[Bead store, Craft store, Jewelry store, Natur...",3.3,18,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Ac...",Open ⋅ Closes 5PM,"[0x80c2c634a0eb5503:0xb47c327f4895fc3b, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


Save the data to a new csv file

In [12]:
la_metadata_df.to_csv('dataset/la_metadata.csv', index=False)

Get the gmap ids of the restaurants

In [19]:
la_gmaps_ids = la_metadata_df['gmap_id'].values

Load in data from review file in chunks, for each chunk, filter out reviews that has the same gmap id as the restaurants in Los Angeles

In [27]:
# Very big file, so we load it in chunks
total_lines = 70_529_977
chunk_size = 1_000_000

review_df = pd.read_json(review_file, lines=True, encoding='utf-8', chunksize=chunk_size)
la_review_df = pd.DataFrame()

for data in tqdm(review_df, total=total_lines//chunk_size + 1):
    indices = data['gmap_id'].isin(la_gmaps_ids)
    la_review_df = pd.concat([la_review_df, data[indices]], ignore_index=True)

  0%|          | 0/71 [00:00<?, ?it/s]

100%|██████████| 71/71 [15:19<00:00, 12.95s/it]


4953122 reviews in total

In [31]:
len(la_review_df), la_review_df.head()

(4953122,
         user_id               name           time  rating  \
 0  1.131656e+20      Michael Rizal  1599164133778     5.0   
 1  1.012264e+20  Faranak Rafizadeh  1618261672851     5.0   
 2  1.111677e+20       Javier Perez  1524515066787     5.0   
 3  1.162309e+20            Luis P.  1499739139293     5.0   
 4  1.041817e+20     His Mama Cakez  1621410940638     3.0   
 
                                                 text  pics  resp  \
 0  Great company. Amazing customer service and th...  None  None   
 1                               Nice people\nhelpful  None  None   
 2                                               None  None  None   
 3                                               None  None  None   
 4                                               None  None  None   
 
                                  gmap_id  
 0  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9  
 1  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9  
 2  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9  
 3  0x80c2c98c0e3c16fd:0

Sanitize data for storing

In [32]:
# Sanitize the data (remove |)
for col in la_review_df.columns:
    if la_review_df[col].dtype == 'object':
        la_review_df[col] = la_review_df[col].str.replace('|', '')

Discard all data that has no text or rating

In [43]:
# remove rows with empty rating
la_review_df = la_review_df[la_review_df['rating'].notnull()]
# remove rows with empty text
la_review_df = la_review_df[la_review_df['text'].notnull()]

Check if any other columns have missing data

In [44]:
# check any other columns with missing values
la_review_df.isnull().sum()

user_id          0
name             0
time             0
rating           0
text             0
pics       2759425
resp       2759425
gmap_id          0
dtype: int64

Great! We don't need pics and resp anyways

In [45]:
len(la_review_df)

2759425

In [47]:
la_review_df.to_csv('dataset/la_review.csv', index=False, sep='|', encoding='utf-8', escapechar='\\')

Let's load in the two data again, and prepare to transform them to a huggingface dataset

In [24]:
la_metadata = pd.read_csv('dataset/la_metadata.csv')

la_review = pd.read_csv('dataset/la_review.csv', sep='|', encoding='utf-8', escapechar='\\')

/tmp/ipykernel_40172/485231083.py:3: DtypeWarning: Columns (0,1,3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  la_review = pd.read_csv('dataset/la_review.csv', sep='|', encoding='utf-8', escapechar='\\')


In [25]:
def split_categories(categories):
    categories = categories[1:-1]
    return [c[1:-1] for c in categories.split(', ')]

la_metadata.dropna(subset=['category'], inplace=True)
la_metadata['category'] = la_metadata['category'].apply(split_categories)

In [26]:
list(la_metadata['category'])

[['Textile exporter'],
 ['Korean restaurant'],
 ['Fabric store'],
 ['Fabric store'],
 ['Bead store',
  'Craft store',
  'Jewelry store',
  'Natural stone supplier',
  'Wholesaler'],
 ['Jewelry store'],
 ['Hair salon'],
 ['Architect'],
 ['Hiking area'],
 ['Student housing center'],
 ['Sports bar', 'Bar'],
 ['Restaurant'],
 ['Event technology service'],
 ['Chauffeur service',
  'Airport shuttle service',
  'Limousine service',
  'Taxi service'],
 ['Bar & grill', 'American restaurant', 'Bar'],
 ['Art restoration service'],
 ['Video editing service', 'Video duplication service'],
 ['Store'],
 ['Construction company'],
 ['Stone supplier',
  'Building materials store',
  'Granite supplier',
  'Marble supplier',
  'Tile store'],
 ['Beauty salon'],
 ['Beauty salon'],
 ['Beauty salon', 'Hair salon'],
 ['Hair salon'],
 ['Recruiter'],
 ['Neurosurgeon', 'Doctor', 'Medical clinic', 'Surgeon'],
 ['Candle store', 'Religious goods store'],
 ['Accountant', 'Certified public accountant', 'Tax preparatio

In [31]:
# find all indices with restaurants in categories

def find_restaurant(categories):
    for c in categories:
        if 'Restaurant' in c:
            return True
    return False

indices = la_metadata['category'].apply(find_restaurant)
la_metadata = la_metadata[indices]

len(la_metadata)

3350

In [32]:
la_metadata.columns, la_review.columns

(Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
        'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
        'state', 'relative_results', 'url'],
       dtype='object'),
 Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object'))

In [33]:
la_review['gmap_id']

0          0x80c2c98c0e3c16fd:0x29ec8a728764fdf9
1          0x80c2c98c0e3c16fd:0x29ec8a728764fdf9
2          0x80c2c778e3b73d33:0xbdc58662a4a97d49
3          0x80c2c778e3b73d33:0xbdc58662a4a97d49
4          0x80c2c778e3b73d33:0xbdc58662a4a97d49
                           ...                  
2870506    0x80c2bfe2e6ba6721:0x9c0445d217b4938f
2870507    0x80c2bfe2e6ba6721:0x9c0445d217b4938f
2870508    0x80c2bfe2e6ba6721:0x9c0445d217b4938f
2870509    0x80c2bfe2e6ba6721:0x9c0445d217b4938f
2870510    0x80c2bfe2e6ba6721:0x9c0445d217b4938f
Name: gmap_id, Length: 2870511, dtype: object

In [34]:
la_metadata.drop(columns=['name', 'address', 'description', 'MISC', 'state', 'relative_results', 'url', 'hours'], inplace=True)
la_review.drop(columns=['user_id', 'name', 'time', 'pics', 'resp'], inplace=True)

In [35]:
#use gmap_id as index
la_metadata.set_index('gmap_id', inplace=True)
la_review.set_index('gmap_id', inplace=True)

In [36]:
# merge the two dataframes
la_df = la_metadata.join(la_review, how='inner')

In [37]:
la_df = la_df[la_df['text'].notnull()]
la_df.reset_index(inplace=True)
la_df.drop(columns=['gmap_id'], inplace=True)

In [38]:
la_df

,latitude,longitude,category,avg_rating,num_of_reviews,price,rating,text
0,34.000181,-118.441249,[Restaurant],4.2,13,NaN,2.0,This is not a cocktail bar. They only serve be...
1,34.000181,-118.441249,[Restaurant],4.2,13,NaN,4.0,Bare bones. Mexican. Pool. $5 modelo. It ain't...
2,34.000181,-118.441249,[Restaurant],4.2,13,NaN,4.0,(Translated by Google) I like the place becaus...
3,34.000181,-118.441249,[Restaurant],4.2,13,NaN,2.0,This is not a cocktail bar. They only serve be...
4,34.000181,-118.441249,[Restaurant],4.2,13,NaN,4.0,Bare bones. Mexican. Pool. $5 modelo. It ain't...
...,...,...,...,...,...,...,...,...
620812,34.051215,-118.264683,"[Thai restaurant, Cafe, Restaurant]",4.6,78,$,5.0,Great food
620813,34.051215,-118.264683,"[Thai restaurant, Cafe, Restaurant]",4.6,78,$,5.0,Great experience!!!
620814,34.051215,-118.264683,"[Thai restaurant, Cafe, Restaurant]",4.6,78,$,4.0,Good Tasting food!!!
620815,34.051215,-118.264683,"[Thai restaurant, Cafe, Restaurant]",4.6,78,$,4.0,Quick food options


In [39]:
la_df.to_csv('dataset/la_df.csv', index=False, sep='|', encoding='utf-8', escapechar='\\')

Analyzing 2.8 million reviews is definitely way too much for my computer to handle. So let's only choose 3000 restuarnts to analyze.

In [44]:
la_metadata_mini = la_metadata.sample(1500)
la_review_mini = la_review[la_review.index.isin(la_metadata_mini.index)]

In [45]:
len(la_review_mini)

295399

Let's also drop out the review that is too short, as they aren't that helpful.

In [46]:
# make la_review_mini its own dataframe
short_review_indices = la_review_mini['text'].str.len() < 100
la_review_mini['text'][short_review_indices] = None

/tmp/ipykernel_40172/3326787054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_review_mini['text'][short_review_indices] = None


In [47]:
len(la_review_mini) - la_review_mini['text'].isnull().sum()

120125

There is probably a better way to gather high-quality reviews. But I think some low quality reviews are also helpful for representating the accurate distribution of the reviews.

In [48]:
la_mini_df = la_metadata_mini.join(la_review_mini, how='inner')

la_mini_df.to_csv('dataset/la_mini_df.csv', index=False, sep='|', encoding='utf-8', escapechar='\\')